Importando as bibliotecas que seão usadas

In [ ]:
###!pip install --upgrade scikit-learn==0.20.3

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

%load_ext google.colab.data_table
#pd.set_option ('display.max_rows', 99999)

###Carregando o conjunto de dados
Usamos a função do pandas para leitura do arquivos CSV.
Dataset do UCI sobre câncer de mama: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer

# Descrição das colunas (variáveis/atributos/features) do conjunto de dados

Attribute Information

    Class: no-recurrence-events, recurrence-events
    Age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
    Menopause: lt40, ge40, premeno
    Tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.
    Inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.
    Nodes-caps: yes, no.
    Deg-malig: 1, 2, 3.
    Breast: letf, right.
    Breast-quad: left-up, left-low, right-up,right-low, central
    Irradiat: yes, no.
    


In [ ]:
dados = pd.read_csv("https://raw.githubusercontent.com/digiampietri/basesDeDados/main/breast-cancer.csv", names=['Class', 'Age', 'Menopause', 'Tumor-size', 'Inv-nodes','Nodes-caps', 'Deg-malig', 'Breast', 'Breast-quad', 'Irradiat'])

In [ ]:

dados.head()

,Class,Age,Menopause,Tumor-size,Inv-nodes,Nodes-caps,Deg-malig,Breast,Breast-quad,Irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [ ]:
dados.groupby('Class').count()

,Age,Menopause,Tumor-size,Inv-nodes,Nodes-caps,Deg-malig,Breast,Breast-quad,Irradiat
Class,,,,,,,,,
no-recurrence-events,201,201,201,201,201,201,201,201,201
recurrence-events,85,85,85,85,85,85,85,85,85


In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Class        286 non-null    object
 1   Age          286 non-null    object
 2   Menopause    286 non-null    object
 3   Tumor-size   286 non-null    object
 4   Inv-nodes    286 non-null    object
 5   Nodes-caps   286 non-null    object
 6   Deg-malig    286 non-null    int64 
 7   Breast       286 non-null    object
 8   Breast-quad  286 non-null    object
 9   Irradiat     286 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.5+ KB


In [ ]:
#Convertendo as características para características numéricas

#Class: Classe [no-recurrence-events, recurrence-events]
dados['Class'] = np.where(dados['Class'] == "no-recurrence-events", 0, dados['Class'])
dados['Class'] = np.where(dados['Class'] == "recurrence-events", 1, dados['Class'])

#Menopause: Indicação do paciente pré ou pós-menopausa [premeno: antes da menopausa, lt40: antes dos 40, ge40: depois dos 40]
dados['Menopause'] = np.where(dados['Menopause'] == "premeno", 0, dados['Menopause'])
dados['Menopause'] = np.where(dados['Menopause'] == "lt40", 1, dados['Menopause'])
dados['Menopause'] = np.where(dados['Menopause'] == "ge40", 2, dados['Menopause'])

#Node-caps: Penetração do tumor na cápsula do linfonodo [no, yes]
dados['Nodes-caps'] = np.where(dados['Nodes-caps'] == "no", 0, dados['Nodes-caps'])
dados['Nodes-caps'] = np.where(dados['Nodes-caps'] == "?", 0.5, dados['Nodes-caps'])
dados['Nodes-caps'] = np.where(dados['Nodes-caps'] == "yes", 1, dados['Nodes-caps'])

#Deg-malig: Grau de malignidade do tumor

#Breast: Lado da mama [left: Esquerda, Right: Direita]
dados['Breast'] = np.where(dados['Breast'] == "left", 0, dados['Breast'])
dados['Breast'] = np.where(dados['Breast'] == "right", 1, dados['Breast'])

#Breast-quad: Quadrante da mama afetado considerando o bico como o centro [left-up, left-low, central, right-up,right-low]
dados['Breast-quad'] = np.where(dados['Breast-quad'] == "left_up", 1, dados['Breast-quad'])
dados['Breast-quad'] = np.where(dados['Breast-quad'] == "left_low", 2, dados['Breast-quad'])
dados['Breast-quad'] = np.where(dados['Breast-quad'] == "?", 3, dados['Breast-quad'])
dados['Breast-quad'] = np.where(dados['Breast-quad'] == "central", 3, dados['Breast-quad'])
dados['Breast-quad'] = np.where(dados['Breast-quad'] == "right_up", 4, dados['Breast-quad'])
dados['Breast-quad'] = np.where(dados['Breast-quad'] == "right_low",5, dados['Breast-quad'])

#Irradiat: Histórico de radioterapia [no, yes]
dados['Irradiat'] = np.where(dados['Irradiat'] == "no", 0, dados['Irradiat'])
dados['Irradiat'] = np.where(dados['Irradiat'] == "yes", 1, dados['Irradiat'])

#Age: Idade [10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99]
dados['Age'] = np.where(dados['Age'] == "10-19", 1, dados['Age'])
dados['Age'] = np.where(dados['Age'] == "20-29", 2, dados['Age'])
dados['Age'] = np.where(dados['Age'] == "30-39", 3, dados['Age'])
dados['Age'] = np.where(dados['Age'] == "40-49", 4, dados['Age'])
dados['Age'] = np.where(dados['Age'] == "50-59", 5, dados['Age'])
dados['Age'] = np.where(dados['Age'] == "60-69", 6, dados['Age'])
dados['Age'] = np.where(dados['Age'] == "70-79", 7, dados['Age'])
dados['Age'] = np.where(dados['Age'] == "80-89", 8, dados['Age'])
dados['Age'] = np.where(dados['Age'] == "90-99", 9, dados['Age'])

#Tumor-size: Diâmetro do tumor (mm) [0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59]
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "0-4", 1, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "5-9", 2, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "10-14", 3, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "15-19", 4, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "20-24", 5, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "25-29", 6, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "30-34", 7, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "35-39", 8, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "40-44", 9, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "45-49", 10, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "50-54", 11, dados['Tumor-size'])
dados['Tumor-size'] = np.where(dados['Tumor-size'] == "55-59", 12, dados['Tumor-size'])

#Inv-nodes: Número de linfonodos axilares [0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39]
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "0-2", 1, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "3-5", 2, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "6-8", 3, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "9-11", 4, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "12-14", 5, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "15-17", 6, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "18-20", 7, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "21-23", 8, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "24-26", 9, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "27-29", 10, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "30-32", 11, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "33-35", 12, dados['Inv-nodes'])
dados['Inv-nodes'] = np.where(dados['Inv-nodes'] == "36-39", 13, dados['Inv-nodes'])



In [ ]:
dados[:500]

,Class,Age,Menopause,Tumor-size,Inv-nodes,Nodes-caps,Deg-malig,Breast,Breast-quad,Irradiat
0,0,3,0,7,1,0,3,0,2,0
1,0,4,0,5,1,0,2,1,4,0
2,0,4,0,5,1,0,2,0,2,0
3,0,6,2,4,1,0,2,1,1,0
4,0,4,0,1,1,0,2,1,5,0
...,...,...,...,...,...,...,...,...,...,...
281,1,3,0,7,1,0,2,0,1,0
282,1,3,0,5,1,0,3,0,1,1
283,1,6,2,5,1,0,1,1,1,0
284,1,4,2,7,2,0,3,0,2,0


##Seleção de Características (features)

In [ ]:
feature_cols = ['Age','Menopause','Tumor-size','Inv-nodes','Nodes-caps','Deg-malig','Breast','Breast-quad','Irradiat']
X = dados[feature_cols]
Y = dados.Class

In [ ]:
Y
Y=Y.astype('int')

##Separação do Conjunto de Dados: Treinamento e Teste

Separado em 3 partes = 2 de treinamento e 1 de teste


In [ ]:
from pandas.core.common import random_state
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=1)

x_train1, x_train2, y_train1, y_train2 = train_test_split(x_train, y_train, test_size=0.5, shuffle=False)



In [ ]:
print('Tamanho do conjunto de treinamento: %i' %x_train.shape[0])
print('Tamanho do conjunto de treinamento1: %i' %x_train1.shape[0])
print('Tamanho do conjunto de treinamento2: %i' %x_train2.shape[0])
print('Tamanho do conjunto de teste: %i' %x_test.shape[0])

Tamanho do conjunto de treinamento: 228
Tamanho do conjunto de treinamento1: 114
Tamanho do conjunto de treinamento2: 114
Tamanho do conjunto de teste: 58


##Treinando os Modelos Explicáveis
1a) Árvore de Decisão para classificar o conjunto de testes (treinada com o conjunto de treinamento 1);

2a) Árvore de Decisão para classificar o conjunto de testes (treinada com o conjunto de treinamento 2);

In [ ]:
classificadorExplicavel1 = DecisionTreeClassifier(criterion="entropy", max_depth=3)
classificadorExplicavel2 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

modeloExplicavelOriginal = classificadorExplicavel1.fit(x_train1,y_train1)

modeloExplicavel2 = classificadorExplicavel2.fit(x_train2,y_train2)


predicaoExplicavel = modeloExplicavelOriginal.predict(x_test)

predicaoExplicavel2 = modeloExplicavel2.predict(x_test)



## Construindo (ou Treinando) os Modelos (parte 2)

Iremos instanciar e treinar os modelos caixa-preta a serem utilizados no laço.

Cada modelo caixa-preta é treinado uma única vez.

**Caixa-preta** (treinado com o conjunto de treinamento 1) para classificar o conjunto de testes, mas também para classificar o conjunto de treinamento 2 (para produzir dados a serem usados pelo modelo explicável [para explicar o caixa-preta]);

**Criamos um arranjo de modelos** para ser utilizado em um laço nos testes que seguem.


In [ ]:
from sklearn.dummy import DummyClassifier # Este classificador será usado como um baseline

# Modelos que poderão ser usados como "caixa-preta"
from sklearn.ensemble import RandomForestClassifier # Floresta Aleatória
from sklearn.linear_model import LogisticRegression # Regressão Logística
from sklearn.neural_network import MLPClassifier # Rede Neural MLP
from sklearn.naive_bayes import GaussianNB # Classificador Baysiano que utiliza distribuição Gaussiana
from sklearn.neighbors import KNeighborsClassifier # KNN - K-Vizinhos mais próximos
from sklearn import svm # SVM: Máquina de Vetores de Suporte


### Estes classificadores serão tratados como 'caixa-preta'
classificadorCaixaPreta1 = DummyClassifier(strategy='most_frequent', random_state=42, constant = None)
classificadorCaixaPreta2 = DecisionTreeClassifier()
classificadorCaixaPreta3 = RandomForestClassifier()
classificadorCaixaPreta4 = svm.SVC()
classificadorCaixaPreta5 = svm.SVC(kernel='poly')
classificadorCaixaPreta6 = LogisticRegression(max_iter=5000)
classificadorCaixaPreta7 = MLPClassifier(max_iter=5000)
classificadorCaixaPreta8 = GaussianNB()
classificadorCaixaPreta9 = KNeighborsClassifier(n_neighbors = 10)

# Criando um arranjo com o nome dos modelos (para aparecer impresso nos resultados)
nomes = ["Dummy", "Árvore de Decisão", "Floresta Aleatoria", "SVM (linear)", "SVM (polinomial)", "Regressão Logística", "Multilayer perceptron", "Gaussian Naive Bayes", "KNN"]


# Criando modelos para os classifidores caixa-preta
# Usando o conjunto de treinamento 1 e a respectiva variável alvo
modeloCaixaPreta1 = classificadorCaixaPreta1.fit(x_train1,y_train1)
modeloCaixaPreta2 = classificadorCaixaPreta2.fit(x_train1,y_train1)
modeloCaixaPreta3 = classificadorCaixaPreta3.fit(x_train1,y_train1)
modeloCaixaPreta4 = classificadorCaixaPreta4.fit(x_train1,y_train1)
modeloCaixaPreta5 = classificadorCaixaPreta5.fit(x_train1,y_train1)
modeloCaixaPreta6 = classificadorCaixaPreta6.fit(x_train1,y_train1)
modeloCaixaPreta7 = classificadorCaixaPreta7.fit(x_train1,y_train1)
modeloCaixaPreta8 = classificadorCaixaPreta8.fit(x_train1,y_train1)
modeloCaixaPreta9 = classificadorCaixaPreta9.fit(x_train1,y_train1)


# Criando um arranjo dos modelos caixa-preta criados
modelosCaixaPreta = [modeloCaixaPreta1, modeloCaixaPreta2, modeloCaixaPreta3, modeloCaixaPreta4, modeloCaixaPreta5, modeloCaixaPreta6, modeloCaixaPreta7, modeloCaixaPreta8, modeloCaixaPreta9]

## Avaliação dos Modelos
Os modelos são avaliados em relação à acurácia, precisão, revocação e medida F1.

Os itens **a** e **b** são relacionados unicamente com o modelo explicável e por isso são executados à parte.

**O foco principal é analisar:**

**[a] Desempenho do modelo 'explicável' 1 (treinado usando o conjunto de treinamento 1);**


[b] Desempenho do modelo 'explicável' 2 (treinado usando o conjunto de treinamento 2);

**[1] Desempenho do modelo 'caixa-preta';**

[2] Desempenho do modelo 'explicável' 1 "explicando" o modelo 'caixa-preta';

[3] Desempenho do modelo 'explicável' 2 (treinamento com variável alvo original) "explicando" o modelo 'caixa-preta';

**[4] Desempenho do modelo 'explicável' 3 (treinamento com saída do caixa-preta) "explicando" o modelo 'caixa-preta;**

**[5] Desempenho do modelo 'explicável' 3 (treinamento com saída do caixa-preta), mas aplicado à variável alvo do conjunto de testes.**

In [ ]:
print('[a] Resultados para Modelo Explicável (treinado com o conjunto de treinamento 1)')
print("Acurácia:",metrics.accuracy_score(y_test, predicaoExplicavel))
report=metrics.classification_report(y_test,predicaoExplicavel)
print(report)
print(confusion_matrix(y_test, predicaoExplicavel))

print('\n[b] Resultados para Modelo Explicável 2 (treinado com o conjunto de treinamento 2)')
print("Acurácia:",metrics.accuracy_score(y_test, predicaoExplicavel2))
report=metrics.classification_report(y_test,predicaoExplicavel2)
print(report)
print(confusion_matrix(y_test, predicaoExplicavel2))

[a] Resultados para Modelo Explicável (treinado com o conjunto de treinamento 1)
Acurácia: 0.7068965517241379
              precision    recall  f1-score   support

           0       0.74      0.88      0.80        40
           1       0.55      0.33      0.41        18

    accuracy                           0.71        58
   macro avg       0.65      0.60      0.61        58
weighted avg       0.68      0.71      0.68        58

[[35  5]
 [12  6]]

[b] Resultados para Modelo Explicável 2 (treinado com o conjunto de treinamento 2)
Acurácia: 0.7068965517241379
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        40
           1       1.00      0.06      0.11        18

    accuracy                           0.71        58
   macro avg       0.85      0.53      0.47        58
weighted avg       0.79      0.71      0.60        58

[[40  0]
 [17  1]]


## Construindo (ou Treinando) os Modelos (parte 3)

Dentro do laço um novo modelo é treinado:

**Árvore de Decisão** para explicar o modelo caixa-preta(treinada com o conjunto de treinamento 2, mas usando como alvo a classificação produzida pelo modelo caixa-preta).

Este modelo também será testado para explicar a saída da Floresta Aleatória, mas também para classificar a variável alvo do conjunto de testes.

## Avaliação dos Modelos
Os modelos são avaliados em relação à acurácia, precisão, revocação e medida F1.

** Neste bloco serão executados os testes que envolvem os modelos caixa-preta. ** Estão destacados os principais resultados para a pesquisa, porém os cinco resultados serão produzidos.


**[1] Desempenho do modelo 'caixa-preta';**

[2] Desempenho do modelo 'explicável' 1 "explicando" o modelo 'caixa-preta';

[3] Desempenho do modelo 'explicável' 2 (treinamento com variável alvo original) "explicando" o modelo 'caixa-preta';

**[4] Desempenho do modelo 'explicável' 3 (treinamento com saída do caixa-preta) "explicando" o modelo 'caixa-preta;**

**[5] Desempenho do modelo 'explicável' 3 (treinamento com saída do caixa-preta), mas aplicado à variável alvo do conjunto de testes.**

In [ ]:
cont = 0
for m in modelosCaixaPreta:

  print("\n############################ " + nomes[cont] + " ############################\n")
  cont = cont + 1
  classificadorExplicavel3 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

  # Realizando a predição inicial (caixa-preta para o teste)
  predicaoCaixaPreta = m.predict(x_test)
  # Produzindo as predições da variável alvo que serão usadas para explicar o caixa-preta
  predicaoCaixaPreta2a = m.predict(x_train2)

  # Treinando o modelo explicativo, com base na predição do modelo caixa-preta
  modeloExplicandoCaixaPreta = classificadorExplicavel3.fit(x_train2,predicaoCaixaPreta2a)

  # Realizando as predições com base no que foi aprendido com o modelo caixa-preta
  predicaoExplicandoCaixaPreta = modeloExplicandoCaixaPreta.predict(x_test)

  print('\n[1] Resultados para o Modelo Caixa-Preta')
  print("Acurácia:",metrics.accuracy_score(y_test, predicaoCaixaPreta))
  report=metrics.classification_report(y_test,predicaoCaixaPreta)
  print(report)
  print(confusion_matrix(y_test, predicaoCaixaPreta))


  print('\n[2] Resultados para o Modelo Explicável 1 - comparando saída com a do caixa preta e não com o target original ("explicando" o caixa-preta)')
  print("Acurácia:",metrics.accuracy_score(predicaoCaixaPreta, predicaoExplicavel))
  report=metrics.classification_report(predicaoCaixaPreta,predicaoExplicavel)
  print(report)
  print(confusion_matrix(predicaoCaixaPreta, predicaoExplicavel))

  print('\n[3] Resultados para o Modelo Explicável 2 (treinado com os dados de treinamento 2, mas comparando saída com a do caixa preta e não com o target original ["explicando" o caixa-preta])')
  print("Acurácia:",metrics.accuracy_score(predicaoCaixaPreta, predicaoExplicavel2))
  report=metrics.classification_report(predicaoCaixaPreta,predicaoExplicavel2)
  print(report)
  print(confusion_matrix(predicaoCaixaPreta, predicaoExplicavel2))

  print('\n[4] Resultados para o Modelo Explicável explicando o Caixa-Preta (este é o único modelo treinado com esta finalidade)')
  print("Acurácia:",metrics.accuracy_score(predicaoCaixaPreta, predicaoExplicandoCaixaPreta))
  report=metrics.classification_report(predicaoCaixaPreta,predicaoExplicandoCaixaPreta)
  print(report)
  print(confusion_matrix(predicaoCaixaPreta, predicaoExplicandoCaixaPreta))

  print('\n[5] Resultados do Modelo Explicável treinado com a saída do Caixa-Preta e aplicado aos dados de teste')
  print("Acurácia:",metrics.accuracy_score(y_test, predicaoExplicandoCaixaPreta))
  report=metrics.classification_report(y_test,predicaoExplicandoCaixaPreta)
  print(report)
  print(confusion_matrix(y_test, predicaoExplicandoCaixaPreta))

  print("\n\n\n")


############################ Dummy ############################


[1] Resultados para o Modelo Caixa-Preta
Acurácia: 0.6896551724137931
              precision    recall  f1-score   support

           0       0.69      1.00      0.82        40
           1       0.00      0.00      0.00        18

    accuracy                           0.69        58
   macro avg       0.34      0.50      0.41        58
weighted avg       0.48      0.69      0.56        58

[[40  0]
 [18  0]]

[2] Resultados para o Modelo Explicável 1 - comparando saída com a do caixa preta e não com o target original ("explicando" o caixa-preta)
Acurácia: 0.8103448275862069
              precision    recall  f1-score   support

           0       1.00      0.81      0.90        58
           1       0.00      0.00      0.00         0

    accuracy                           0.81        58
   macro avg       0.50      0.41      0.45        58
weighted avg       1.00      0.81      0.90        58

[[47 11]
 [ 0  0]]

[

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

              precision    recall  f1-score   support

           0       0.93      0.83      0.88        48
           1       0.47      0.70      0.56        10

    accuracy                           0.81        58
   macro avg       0.70      0.77      0.72        58
weighted avg       0.85      0.81      0.82        58

[[40  8]
 [ 3  7]]

[5] Resultados do Modelo Explicável treinado com a saída do Caixa-Preta e aplicado aos dados de teste
Acurácia: 0.7758620689655172
              precision    recall  f1-score   support

           0       0.81      0.88      0.84        40
           1       0.67      0.56      0.61        18

    accuracy                           0.78        58
   macro avg       0.74      0.72      0.72        58
weighted avg       0.77      0.78      0.77        58

[[35  5]
 [ 8 10]]





############################ SVM (linear) ############################


[1] Resultados para o Modelo Caixa-Preta
Acurácia: 0.6896551724137931
              precision    r

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat


[1] Resultados para o Modelo Caixa-Preta
Acurácia: 0.6896551724137931
              precision    recall  f1-score   support

           0       0.69      1.00      0.82        40
           1       0.00      0.00      0.00        18

    accuracy                           0.69        58
   macro avg       0.34      0.50      0.41        58
weighted avg       0.48      0.69      0.56        58

[[40  0]
 [18  0]]

[2] Resultados para o Modelo Explicável 1 - comparando saída com a do caixa preta e não com o target original ("explicando" o caixa-preta)
Acurácia: 0.8103448275862069
              precision    recall  f1-score   support

           0       1.00      0.81      0.90        58
           1       0.00      0.00      0.00         0

    accuracy                           0.81        58
   macro avg       0.50      0.41      0.45        58
weighted avg       1.00      0.81      0.90        58

[[47 11]
 [ 0  0]]

[3] Resultados para o Modelo Explicável 2 (treinado com os dados de

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMe